In [2]:
from mesa import Agent,Model
from mesa.time import RandomActivation
from mesa.space import Grid
from matplotlib import pyplot as plt
from mesa.datacollection import DataCollector

from mesa.batchrunner import BatchRunner

In [12]:


class Students(Agent):
    def __init__(self,unique_id,condition,day,recovered,model):
        super().__init__(unique_id,model)
        self.condition = condition
        self.day =day
        self.recovered = recovered

#     def p(self):
#         print('id:'+str(agent.unique_id)+' my condition: '+str(agent.condition))

    
    def step(self):
        if self.condition == "Infected":
            possible_neighbour = self.model.grid.get_neighbors(self.pos,moore = False)
            for i in possible_neighbour:               
                if i.condition == "Suseptible" and self.random.random() <  self.model.prob and i.recovered != "1":
                    i.condition = "Infected"
                    i.day = 0
            self.day = self.day +1
            if self.day  == 3 :
                self.condition = "Recovered"
                self.recovered = "1"
        
            
class Disease_spread(Model):
    
    
    def place_agent(self):
        for a in self.schedule.agents:
            x = self.random.randrange(20)
            y = self.random.randrange(20)
            self.grid.place_agent(a,(x,y))

    def __init__(self,N= 100,probability = 0.4):
        self.N = N 
        self.prob = probability
        self.schedule =RandomActivation(self)
        self.grid = Grid(20,20,False)
        self.condition0 = "Suseptible"
        self.condition1 = "Infected"
        self.condition2 = "Recovered"
        self.day =0
        self.recovered = "0"
        self.datacollector = DataCollector(
        
        {
                "Suseptible": lambda m: self.count_d(m, "Suseptible"),
                "Infected": lambda m: self.count_d(m, "Infected"),
                "Recovered": lambda m: self.count_d(m, "Recovered"),
        }
        )
        
        
        count = 0
        for i in range(self.N):
            if count < 10:
                a = Students(i,self.condition1,self.day,self.recovered,self)
                count = count+1
            else: a = Students(i,self.condition0,self.day,self.recovered,self)
            self.schedule.add(a)
        self.place_agent()
        self.running = True
            
    def step(self):
        self.place_agent()
        self.schedule.step()
        self.datacollector.collect(self)
        if self.count_d(self,"Infected") == 0:
            self.running = False
    
    @staticmethod       
    def count_d(model,agent_condition):
        c = 0 
        for agents in model.schedule.agents:
            if agents.condition == agent_condition:
                c =c  +1
        return c
        

In [13]:
from mesa.visualization.modules import CanvasGrid,ChartModule
#Number of agents slider
from mesa.visualization.UserParam import UserSettableParameter

from mesa.visualization.ModularVisualization import ModularServer 

In [14]:
COLORS = {"Suseptible": "#00AA00", "Infected": "#880000", "Recovered": "#000000"}

def Disease_spread_portrayal(agent):
    if agent is None:
        return
    portrayal = {"Shape": "rect","w": 1, "h": 1,"Filled": "true", "Layer": 0}
    (x,y) = agent.pos
    portrayal["x"] = x
    portrayal["y"] = y
    portrayal["Color"] = COLORS[agent.condition]
    return portrayal

canvas_element = CanvasGrid(
    Disease_spread_portrayal,20,20,500,500

)

tree_chart = ChartModule(
    [{"Label": label, "Color": color} for (label, color) in COLORS.items()]
)   


server = ModularServer(
    Disease_spread,[canvas_element,tree_chart],"Disease Progresition",{"N":100,"probability" : 0.6}
)






In [16]:
server.port = 8540
server.launch()

Interface starting at http://127.0.0.1:8540


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}


In [61]:
model = Disease_spread(100,0.4)

In [62]:
model.step()

In [63]:
# model.step()
c = 0 
for agents in model.schedule.agents:
    if agents.condition == "Suseptible":
        c =c  +1
print(c)

84


In [64]:
for agent in model.schedule.agents:
    print('id:'+str(agent.unique_id)+' my condition: '+str(agent.condition)+'  day:  '+str(agent.day)+'recovered:'+str(agent.recovered))

id:0 my condition: Infected  day:  1recovered:0
id:1 my condition: Infected  day:  1recovered:0
id:2 my condition: Infected  day:  1recovered:0
id:3 my condition: Infected  day:  1recovered:0
id:4 my condition: Infected  day:  1recovered:0
id:5 my condition: Infected  day:  1recovered:0
id:6 my condition: Infected  day:  1recovered:0
id:7 my condition: Infected  day:  1recovered:0
id:8 my condition: Infected  day:  1recovered:0
id:9 my condition: Infected  day:  1recovered:0
id:10 my condition: Suseptible  day:  0recovered:0
id:11 my condition: Suseptible  day:  0recovered:0
id:12 my condition: Suseptible  day:  0recovered:0
id:13 my condition: Suseptible  day:  0recovered:0
id:14 my condition: Suseptible  day:  0recovered:0
id:15 my condition: Suseptible  day:  0recovered:0
id:16 my condition: Suseptible  day:  0recovered:0
id:17 my condition: Suseptible  day:  0recovered:0
id:18 my condition: Suseptible  day:  0recovered:0
id:19 my condition: Suseptible  day:  0recovered:0
id:20 my c

In [19]:
import numpy as np

condition = np.zeros((model.grid.width, model.grid.height))
for agent in model.schedule.agents:
    x,y = agent.pos
    condi = agent.condition
    condition[x][y] = condi
plt.imshow(condition)
plt.colorbar()
plt.show()

ValueError: could not convert string to float: 'Infected'

In [142]:
for agent in model.schedule.agents:
    print('id:'+str(agent.unique_id)+' my condition: '+str(agent.condition)+'day:'+str(agent.day)+'recovered:'+str(agent.recovered))

id:0 my condition: Infectedday:0recovered:0
id:1 my condition: Infectedday:0recovered:0
id:2 my condition: Infectedday:0recovered:0
id:3 my condition: Infectedday:0recovered:0
id:4 my condition: Infectedday:0recovered:0
id:5 my condition: Infectedday:0recovered:0
id:6 my condition: Infectedday:0recovered:0
id:7 my condition: Infectedday:0recovered:0
id:8 my condition: Infectedday:0recovered:0
id:9 my condition: Infectedday:0recovered:0
id:10 my condition: Suseptibleday:0recovered:0
id:11 my condition: Suseptibleday:0recovered:0
id:12 my condition: Suseptibleday:0recovered:0
id:13 my condition: Suseptibleday:0recovered:0
id:14 my condition: Suseptibleday:0recovered:0
id:15 my condition: Suseptibleday:0recovered:0
id:16 my condition: Suseptibleday:0recovered:0
id:17 my condition: Suseptibleday:0recovered:0
id:18 my condition: Suseptibleday:0recovered:0
id:19 my condition: Suseptibleday:0recovered:0
id:20 my condition: Suseptibleday:0recovered:0
id:21 my condition: Suseptibleday:0recovere

In [86]:
model.step()
c = 0 
for agents in model.schedule.agents:
    if agents.condition == 1:
        c =c  +1
print(c)

0
